## Investigate asyncio for parallel processing to reduce compute time

In [1]:
# General imports
import os
import numpy as np
import matplotlib.pyplot as plt

# Segmentation
from astropy.io import fits
from photutils.segmentation import SourceFinder, make_2dgaussian_kernel, SourceCatalog
from photutils.background import Background2D, MedianBackground
from astropy.convolution import convolve

# grizli
from grizli.model import GrismFLT
import grizli

os.chdir("/Users/keith/astr/research_astr/summer-roman-project/all_chips/fits_scripts")

from roman_fits_utils import fix_hdr, rotate_img
from roman_seg_map import produce_seg_map


os.chdir("/Users/keith/astr/research_astr/all_chip_fits")

In [2]:
# Function for dynamic filename
file = lambda ii: "GRS_FOV0_roll0_dx0_dy0_SCA%i_direct_final.fits" %ii

for ii in range(1, 19):
    filename = file(ii)
    fixed_hdr_name = "fixed_hdr_%i.fits" %ii
    rotated_img_name = "rotated_img_%i.fits" %ii

    hdr_dict = {"INSTRUME": "0;ROMAN",
                "FILTER":   "0;det%i" %ii,
                "CONFFILE": "1;/Users/keith/astr/research_astr/FOV0/Roman.det%i.07242020.conf" %ii}

    fix_hdr(filename, fixed_hdr_name, hdr_dict)

    rotate_img(fixed_hdr_name, rotated_img_name)

print("Fits Files Prepared for modelling")

Fits Files Prepared for modelling


In [3]:

# Altered functions for dynamic filename
file = lambda ii: "rotated_img_%i.fits" %ii
seg_file = lambda ii: "seg_%i.fits" %ii

for ii in range(1, 19):
    filename = file(ii)
    seg_filename = seg_file(ii)

    produce_seg_map(filename, seg_filename)

Deblending:   0%|          | 0/6390 [00:00<?, ?it/s]

Deblending:   0%|          | 0/5663 [00:00<?, ?it/s]

Deblending:   0%|          | 0/1424 [00:00<?, ?it/s]

Deblending:   0%|          | 0/5878 [00:00<?, ?it/s]

Deblending:   0%|          | 0/6136 [00:00<?, ?it/s]

...

Deblending:   0%|          | 0/6266 [00:00<?, ?it/s]

Deblending:   0%|          | 0/2784 [00:00<?, ?it/s]

Deblending:   0%|          | 0/4795 [00:00<?, ?it/s]

Deblending:   0%|          | 0/5437 [00:00<?, ?it/s]

Deblending:   0%|          | 0/4981 [00:00<?, ?it/s]

In [4]:
# Instantiate the GrismFLT objects
pad = 100
sims = {}
for ii in range(1, 19):
    sims["%i_sim" %ii] = GrismFLT(direct_file=file(ii), seg_file=seg_file(ii), pad=pad)



Image cutout: x=slice(0, 4088, None), y=slice(0, 4088, None) [Out of range]
rotated_img_1.fits / Pad ref HDU with 264 pixels
rotated_img_1.fits / blot segmentation seg_1.fits[0]
The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol
Using default C-based coordinate transformation...
Using default C-based coordinate transformation...
Image cutout: x=slice(0, 4088, None), y=slice(0, 4088, None) [Out of range]
rotated_img_2.fits / Pad ref HDU with 264 pixels
rotated_img_2.fits / blot segmentation seg_2.fits[0]
Using default C-based c

In [7]:
sims.keys()

dict_keys(['1_sim', '2_sim', '3_sim', '4_sim', '5_sim', '6_sim', '7_sim', '8_sim', '9_sim', '10_sim', '11_sim', '12_sim', '13_sim', '14_sim', '15_sim', '16_sim', '17_sim', '18_sim'])

In [6]:
# Compute Full Models
det_sizes = {"det1": 77,
            "det2": 80,
            "det3": 88,
            "det4": 180,
            "det5": 195,
            "det6": 209,
            "det7": 279,
            "det8": 302,
            "det9": 328,
            "det10": 1,
            "det11": 2,
            "det12": 6,
            "det13": 30,
            "det14": 18,
            "det15": 2,
            "det16": 78,
            "det17": 49,
            "det18": 22}

for ii, detector in enumerate(det_sizes.keys()):


    if det_sizes[detector] < 77: # This needs to be checked; For detectors with small max dy, what it a good min_size?
        sims["%i_sim" %(ii+1)].compute_full_model(mag_limit=28)
    else:
        sims["%i_sim" %(ii+1)].compute_full_model(mag_limit=28, compute_size=False, size=det_sizes[detector])

KeyError: '0_sim'

In [ ]:
for ii in range(1, 19):
    header = fits.open(file(ii))[0].header
    fits.writeto("full_model_%i.fits" %i, sims["%i_sim" %ii].model[pad:-pad, pad:-pad], header=header, overwrite=True)